In [5]:
from Exercise import Exercise, MarkdownBlock
from config import URL, TOKEN
import json

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams.update({'font.size': 20})

from sklearn.datasets import load_digits
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Exercise.URL = URL
Exercise.TOKEN = TOKEN

In [6]:
from sympy import Rational, Symbol, latex, UnevaluatedExpr

u = lambda x : UnevaluatedExpr(x)

# Helper functions
def explain_add(a, b):
    assert(np.shape(a) == np.shape(b))
    rows, columns = np.shape(a)
    return sp.Matrix([[Symbol(f"({latex(u(a[i,j]))} + {latex(u(b[i,j]))})") for j in range(columns)] for i in range(rows)])

def symbolic_matrix(character, rows, columns):
    return sp.Matrix([[Symbol(f"{{{character}}}_{{{i+1}, {j+1}}}") for j in range(columns)] for i in range(rows)])

def explain_multiply(a, b):
    # #rows in b == #columns in a
    assert(np.shape(a)[1] == np.shape(b)[0])
    rows = np.shape(a)[0]
    columns = np.shape(b)[1]
    result = np.empty(shape=(rows, columns), dtype=object)
    for i in range(rows):
        row = a[i,:]
        for j in range(columns):
            column = b[:,j]
            zipped = zip(row, column)
            mapped = list(map(lambda t: f"{latex(u(t[0]))} \cdot {latex(u(t[1]))}", zipped))
            s = Symbol("") 
            result[i, j] = Symbol(" + ".join(mapped), evaluate=False)
                
    return sp.Matrix(result)

# TODO: fixme for >= 3 dim matrices
# TODO: print intermediate steps at each return
def explain_det(a):
    # square matrix
    assert(np.shape(a)[0] == np.shape(a)[1])
    if np.shape(a)[0] == 1:
        return a[0,0]
    if np.shape(a)[0] == 2:
        return f"{a[0,0]} \cdot {a[1,1]} - {a[0,1]} \cdot {a[0,1]}"
    else:
        return f"{a[0,0]} \cdot {explain_det(a[1:,1:])} - {a[0,1]} \cdot {[[a[1,0], a[1,2]], [a[2,0], a[2,2]]]} + {a[0,2]} \cdot {[[a[1,0], a[1,2]], [a[1,1], a[2,1]]]}"

# A = symbolic_matrix("a", 3, 3)
# e = Exercise(f"${explain_det(A)}$")
# e.display()

In [7]:
a = np.arange(4)
b = -1 * np.arange(1, 5)
a = sp.Matrix(a)
b = sp.Matrix(b)

s = "What is $@a + @b?$"

params = {}
params["a"] = a
params["b"] = b
e = Exercise(MarkdownBlock(s, params))

e.display()

s1 = "Yes, $@a + @b = @c = @d$!"

params["c"] = explain_add(a,b)
params["d"] = a + b
e.add_answer(a + b, True, MarkdownBlock(s1, params))
e.evaluate_answer(params["d"])

e.x = symbolic_matrix("a", 4,1)
e.y = symbolic_matrix("b", 4,1)
e.z = explain_add(e.x, e.y)
default_feedback = "Remember the definition of matrix addition: $@x + @y = @z$"
e.add_default_feedback(MarkdownBlock(default_feedback, vars(e)))
e.evaluate_answer(1)

e.write()
# e.publish()

In [4]:
s = "What is $@a \cdot @b$?"

rows = np.random.randint(1, 6)
columns = np.random.randint(1, 6)

params = {}
params["a"] = sp.Matrix(np.random.randint(5, size=rows*columns).reshape((rows,columns)))
params["b"] = sp.Matrix(np.random.randint(5, size=(2+rows)*columns).reshape((columns,rows+2)))

e = Exercise(MarkdownBlock(s, params))

ans = params["a"] * params["b"]
display(ans)
e.add_answer(params["a"] * params["b"], True, "That's right!")


params = {}
params["x"] = symbolic_matrix("a", rows, columns)
params["y"] = symbolic_matrix("b", columns, rows)
params["z"] = explain_multiply(params["x"], params["y"])
f = """
Remember the definition of matrix multiplication:

$@x \cdot @y = @z$
"""
e.add_default_feedback(MarkdownBlock(f, params))
# e.evaluate_answer(1)


e.display()
e.write()
e.publish()

Matrix([[15, 21, 18]])

Published succesfully, preview at: https://www.mscthesis.nl/preview?id=9bb03fc1-4084-4139-9c89-7325ed4220ee


In [ ]:
def explain_multiply(a, b):
    # #rows in b == #columns in a
    assert(np.shape(a)[1] == np.shape(b)[0])
    rows = np.shape(a)[0]
    columns = np.shape(b)[1]
    result = np.empty(shape=(rows, columns), dtype=object)
    for i in range(rows):
        row = a[i,:]
        for j in range(columns):
            column = b[:,j]
            zipped = zip(row, column)
            mapped = list(map(lambda t: f"{latex(u(t[0]))} \cdot {latex(u(t[1]))}", zipped))
            s = Symbol("") 
            result[i, j] = Symbol(" + ".join(mapped), evaluate=False)
                
    return sp.Matrix(result)

# explain_multiply(np.arange(4).reshape((2,2)), np.arange(4).reshape((2,2)))

In [12]:
v = symbolic_matrix("a", 3, 1)
display(3*u(v))
display(3*v)


3*Matrix([
[{a}_{1, 1}],
[{a}_{2, 1}],
[{a}_{3, 1}]])

Matrix([
[3*{a}_{1, 1}],
[3*{a}_{2, 1}],
[3*{a}_{3, 1}]])

# TODO: meaningfull, contextualized exercises for each of these:
- scalar def.
- vector def.
- matrix def.
- vector indexing
- matrix indexing
- total values in a matrix
- vector dimensions
- matrix dimensions
- special matrices
- transpose
- operations (+, *), linear combinations
- dot product, cross product, norm
- cosine similarity

## Matrix Transpose

In [24]:
m = "Determine $@a^\intercal$?"

a = np.arange(6).reshape((2, 3))

params = {}
params["a"] = sp.Matrix(a)

e = Exercise(MarkdownBlock(m, params))
e.display()

e.add_answer(params["a"].T, True, "Correct!")

e.write()
e.publish()

Published succesfully, preview at: https://www.mscthesis.nl/preview?id=d48e4e02-9004-4a66-8233-bbede217f120


## Matrix Inverse

In [41]:
m = "Determine $@a^{-1}$?"

a = np.arange(4).reshape((2, 2))

params = {}
params["a"] = sp.Matrix(a)

e = Exercise(MarkdownBlock(m, params))
e.display()

e.add_answer(params["a"].inv(), True, "Correct!")

e.write()
e.publish()

Published succesfully, preview at: https://www.mscthesis.nl/preview?id=9936ec26-6a1a-420d-b685-f53902265397


In [61]:
m = "Given $A = @a$, what is $det(A)$?"

a = np.arange(4).reshape((2, 2))
params = {}
params["a"] = sp.Matrix(a)

e = Exercise(MarkdownBlock(m, params))
e.display()

e.add_answer(params["a"].det(), True, "Correct!")

e.write()
e.publish()

Published succesfully, preview at: https://www.mscthesis.nl/preview?id=6e731c50-d2f4-474a-8f59-3f9976d352a7
